# Minimally Sufficient Pandas

This notebook shows a practical overview of [Minimally Sufficient Pandas](https://medium.com/dunder-data/minimally-sufficient-pandas-a8e67f2a2428) post on Medium. Our main goal is reproducing his assumptions in a different dataset.

This dataset can be found in [Ghouls, Goblins, and Ghosts... Boo!](https://www.kaggle.com/c/ghouls-goblins-and-ghosts-boo/) competition hosted on Kaggle.

So... Let's get started!

---

## Pandas is Powerful but Difficult to use
Pandas is the most popular Python library for doing data analysis. While it does offer quite a lot of functionality, it is also regarded as a fairly difficult library to learn well. Some reasons for this include:

* There are often multiple ways to complete common tasks
* There are over 240 DataFrame attributes and methods
* There are several methods that are aliases (reference the same exact underlying code) of each other
* There are several methods that have nearly identical functionality
* There are many tutorials written by different people that show different ways to do the same thing
* There is no official document with guidelines on how to idiomatically complete common tasks
* The official documentation, itself contains non-idiomatic code

## Specific Pandas Examples
We will now cover a series of specific examples within Pandas where multiple approaches exist to complete a task. I will compare and contrast the different approaches and give guidance on which one I prefer. Listed below are the topics I cover.

* Selecting a single column of data
* The deprecated `ix` indexer
* Selection with `at` and `iat`
* `read_csv` vs `read_table` duplication
* `isna` vs `isnull` and `notna` vs `notnull`
* Arithmetic and Comparison Operators and their Corresponding Methods
* Builtin Python functions vs Pandas methods with the same name
* Standardizing `groupby` aggregation
* Handling a MultiIndex
* `pivot` vs `pivot_table`
* The similarity between `melt` and `stack`
* The similarity between `pivot` and `unstack`

## Minimally Sufficient Guiding Principle
The concrete examples were all derived by the following principle:

> If a method does not provide any additional functionality over another method (i.e. its functionality is a subset of another) then it shouldn’t be used. Methods should only be considered if they have some additional, unique functionality.


### Selecting a Single Columns of Data
Selecting a single column of data from a Pandas DataFrame is just about the simplest task you can do and unfortunately, it is here where we first encounter the multiple-choice option that Pandas presents to its users.

You may select a single column as a Series with either the **brackets** or **dot notation**. Let’s read in a small, trivial DataFrame and select a column using both methods.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
names = ['id', 'bone length', 'rotting flesh', 'hair length', 'has soul', 'color', 'type']
df = pd.read_csv('./data/train.csv', names=names, skiprows=1, index_col=0)
df.head()

,bone length,rotting flesh,hair length,has soul,color,type
id,,,,,,
0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,0.467875,0.354330,0.811616,0.791225,black,Ghoul
4,0.776652,0.508723,0.636766,0.884464,black,Ghoul
5,0.566117,0.875862,0.418594,0.636438,green,Ghost


#### Selection with the brackets
Placing a column name in the brackets appended to a DataFrame selects a single column of a DataFrame as a Series.

In [2]:
df['type'].head()

id
0     Ghoul
1    Goblin
2     Ghoul
4     Ghoul
5     Ghost
Name: type, dtype: object

#### Selection with dot notation
Alternatively, you may select a single column with dot notation. Simply, place the name of the column after the dot operator. The output is the exact same as above.

In [3]:
df.type.head()

id
0     Ghoul
1    Goblin
2     Ghoul
4     Ghoul
5     Ghost
Name: type, dtype: object

#### Issues with the dot notation
There are three issues with using dot notation. It doesn't work in the following situations:

* When there are spaces in the column name
* When the column name is the same as a DataFrame method
* When the column name is a variable

#### There are spaces in the column name
If the desired column name has spaces in it, you won't be able to select it with the dot notation. Python uses spaces to separate names and operators and hence will not treat a column name with a space as correct syntax. Let's create this error.

In [4]:
df.has soul

SyntaxError: invalid syntax (<ipython-input-4-78d71ec67497>, line 1)

You can only use the brackets to select columns with spaces.

In [6]:
df['has soul'].head()

id
0    0.781142
1    0.439899
2    0.791225
4    0.884464
5    0.636438
Name: has soul, dtype: float64

#### The column name is the same as a DataFrame method
When a column name and a DataFrame method collide, Pandas will always reference the method and not the column name. For instance, the column name `count` is a method and will be referenced when using dot notation. This actually doesn't produce an error as Python allows you to reference methods without calling them. Let's reference this method now.

In [7]:
target_count = df.type.value_counts().to_frame().rename(columns={'type':'count'})
target_count

,count
Ghoul,129
Goblin,125
Ghost,117


In [8]:
target_count.count

<bound method DataFrame.count of         count
Ghoul     129
Goblin    125
Ghost     117>

The output is going to be very confusing if you haven't encountered it before. Notice at the top it states 'bound method DataFrame.count of'. Python is telling us that this is a method of some DataFrame object. Instead of using the method name, it outputs its official string representation. Many people believe that they've produced some kind of analysis with this result. This isn't true and almost nothing has happened. A reference to the method that outputs the object's representation has been produced. That is all.

Regardless, it's clear that using the dot notation did not select a single column of the DataFrame as a Series. Again, you must use the brackets when selecting a column with the same name as a DataFrame method.

In [9]:
target_count['count']

Ghoul     129
Goblin    125
Ghost     117
Name: count, dtype: int64

#### Guidance: Use the brackets for selecting a column of data
The dot notation provides no additional functionality over the brackets and does not work in all situations. Therefore, I never use it. Its single advantage is three fewer keystrokes.

### The deprecated ix indexer - NEVER USE IT!
Pandas allows you to select rows by either label or integer location. This flexible dual selection capability is a great cause of confusion for begginers. The `ix` indexer was created in the early days of Pandas to select rows and columns by bith label and integer location. This turned out to be quite ambiguous as Pandas row and column names can be both integers and strings.

To make selections explicit, the `loc` and `iloc` indexers were made available. The `loc` indexer selects only by label while the `iloc` indexer selects only by integer location. Although the `ix` indexer was versatile, it has been deprecated in favor of the `loc` and `iloc` indexers.

#### Guidance: Every trace of ix should be removed and replaced with loc or iloc

### Selection with at and iat
Two additional indexers, `at` and `iat`, exist that select a single cell of a DataFrame. These provide a slight performance advantage over their analogous `loc` and `iloc` indexers. But, they introduce the additional burden of having to remember what they do. Also, for the most data analyses, the increase in performance isn't useful unless it's being done at scale. And if the performance truly is an issue, then taking your data out of a DataFrame and into a NumPy array will give you a large performance gain.

#### Performance comparison iloc vs iat vs NumPy
Let's compare the performance of selectiong a single cell with `iloc`, `iat` and a NumPy array. Here we create a NumPy array with 100k rows and 5 columns containing random data. We then create a DataFrame out of it and make the selections.

In [10]:
import numpy as np
a   = np.random.rand(10 ** 5, 5)
df1 = pd.DataFrame(a)

row = 50000
col = 3

In [11]:
%timeit df1.iloc[row, col]

9.99 µs ± 271 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
%timeit df1.iat[row, col]

6.77 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
%timeit a[row, col]

246 ns ± 6.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


While `iat` is a little more than a half as fast as `iloc`, selection with a NumPy array is about 60x as fast. So, if you really had an application that had performance requirements, you should be using NumPy directly and not Pandas.

#### Guidance: Use NumPy arrays if your application relies of performance for selecting a single cell of data and not at or iat

### Method Duplication
There are multiple methods in Pandas that do the exact same thing. Whenever two methods share the same exact underlying functionality, we say that they are **aliases** of each other. Having duplication in a library is completely unnecessary, pollutes the namespace and forces analysts to remember one more bit of information about a library.

This next section covers several instances of duplication along with other instances of methods that are very similar to one another.

### read_csv vs read_table duplication
One example of duplication is with the `read_csv` and `read_table` functions. They both do the same exact thing, read in data from a text file. The only difference is that `read_csv` defaults the delimiter to a comma, while `read_table` uses tab as its default.

Let's verify that `read_csv` and `read_table` are capable of producing the same results. Here we use a sample of the [College Scoreboard Dataset](https://collegescorecard.ed.gov/data/). The `equals` method verifies whether two DataFrames have the exact same values.

In [20]:
import warnings
warnings.simplefilter('ignore')

In [21]:
college = pd.read_csv('./data/college.csv')
college.head()

UNITID    OPEID  OPEID6                               INSTNM        CITY  \
0  100654   100200    1002             Alabama A & M University      Normal   
1  100663   105200    1052  University of Alabama at Birmingham  Birmingham   
2  100690  2503400   25034                   Amridge University  Montgomery   
3  100706   105500    1055  University of Alabama in Huntsville  Huntsville   
4  100724   100500    1005             Alabama State University  Montgomery   

  STABBR         ZIP                                       ACCREDAGENCY  \
0     AL       35762  Southern Association of Colleges and Schools C...   
1     AL  35294-0110  Southern Association of Colleges and Schools C...   
2     AL  36117-3553  Southern Association of Colleges and Schools C...   
3     AL       35899  Southern Association of Colleges and Schools C...   
4     AL  36104-0271  Southern Association of Colleges and Schools C...   

                     INSTURL  \
0              www.aamu.edu/   
1                www.uab.edu   
2  www.amridgeuniversity.edu   
3                www.uah.edu   
4              www.alasu.edu   

                                              NPCURL  SCH_DEG  HCM2  MAIN  \
0  https://galileo.aamu.edu/NetPriceCalculator/np...      NaN     0     1   
1           uab.studentaidcalculator.com/survey.aspx      NaN     0     1   
2                   www2.amridgeuniversity.edu:9091/      NaN     0     1   
3                                    finaid.uah.edu/      NaN     0     1   
4  www.alasu.edu/cost-aid/forms/calculator/index....      NaN     0     1   

   NUMBRANCH  PREDDEG  HIGHDEG  CONTROL  ST_FIPS  REGION  LOCALE  LOCALE2  \
0          1        3        4        1        1       5    12.0      NaN   
1          1        3        4        1        1       5    12.0      NaN   
2          1        3        4        2        1       5    12.0      NaN   
3          1        3        4        1        1       5    12.0      NaN   
4          1        3        4        1        1       5    12.0      NaN   

    LATITUDE  LONGITUDE  CCBASIC  CCUGPROF  CCSIZSET  HBCU  PBI  ANNHI  \
0  34.783368 -86.568502     18.0      10.0      14.0   1.0  0.0    0.0   
1  33.505697 -86.799345     15.0       9.0      15.0   0.0  0.0    0.0   
2  32.362609 -86.174010     20.0       5.0       6.0   0.0  0.0    0.0   
3  34.724557 -86.640449     16.0      15.0      12.0   0.0  0.0    0.0   
4  32.364317 -86.295677     19.0      10.0      14.0   1.0  0.0    0.0   

   TRIBAL  AANAPII HSI  NANTI  MENONLY  WOMENONLY  RELAFFIL  ADM_RATE  \
0     0.0      0.0   0    0.0      0.0        0.0       NaN    0.9027   
1     0.0      0.0   0    0.0      0.0        0.0       NaN    0.9181   
2     0.0      0.0   0    0.0      0.0        0.0      74.0       NaN   
3     0.0      0.0   0    0.0      0.0        0.0       NaN    0.8123   
4     0.0      0.0   0    0.0      0.0        0.0       NaN    0.9787   

   ADM_RATE_ALL  SATVR25  SATVR75  SATMT25  SATMT75  SATWR25  SATWR75  \
0      0.902671    365.0    485.0    360.0    495.0      NaN      NaN   
1      0.918068    440.0    630.0    550.0    740.0      NaN      NaN   
2           NaN      NaN      NaN      NaN      NaN      NaN      NaN   
3      0.812304    550.0    660.0    530.0    670.0      NaN      NaN   
4      0.978661    380.0    485.0    375.0    481.0      NaN      NaN   

   SATVRMID  SATMTMID  SATWRMID  ACTCM25  ACTCM75  ACTEN25  ACTEN75  ACTMT25  \
0     425.0     428.0       NaN     16.0     19.0     14.0     20.0     15.0   
1     535.0     645.0       NaN     21.0     28.0     22.0     31.0     21.0   
2       NaN       NaN       NaN      NaN      NaN      NaN      NaN      NaN   
3     605.0     600.0       NaN     25.0     31.0     25.0     33.0     24.0   
4     433.0     428.0       NaN     16.0     20.0     15.0     19.0     14.0   

   ACTMT75  ACTWR25  ACTWR75  ACTCMMID  ACTENMID  ACTMTMID  ACTWRMID  SAT_AVG  \
0     18.0      NaN      NaN      18.0      17.0      17.0       NaN    9

In [22]:
college2 = pd.read_table('./data/college.csv', delimiter=',')
college.equals(college2)

True

#### Guidance: Only use read_csv to read in delimited text files

### isna() vs isnull() and notna() vs notnull()
The `isna()` and `isnull()` methods both determine whether each value in the DataFrame is missing or not. The result will always be a DataFrame (or Series) of all boolean values.

These methods are exactly the same. We say that one is an **alias** of the other. There is no need for both of them in the library. The `isna()` method was added because the characters `na` are found in other missing value methods such as `dropna()` and `fillna()`. Confunsingly, Pandas uses `NaN`, `None` and `NaT` as missing value representations and not `NA`.

`notna()` and `notnull()` are aliases of each other as well and simply return the opposite of `isna()`. There's no need for both of them.

Let's verify that `isna()` and `isnull()` are aliases.

In [23]:
college_isna   = college.isna()
college_isnull = college.isnull()
college_isna.equals(college_isnull)

True

#### I only use isna and notna
I use the methods that end in `na` to match the names of the other missing values methods `dropna` and `fillna`. 

You can also avoud using `notna` since Pandas provides the inversion operator, `~` to invert boolean DataFrames.

#### Guidance: Use only isna and notna

### Arithmetic and Comparison Operators and their Corresponding Methods
All arithmetic operators have corresponding methods that function similarly.
* `+` - `add`
* `-` - `sub` and `subtract`
* `*` - `mul` and `multiply`
* `/` - `div`, `divide` and `truediv`
* `**` - `pow`
* `//` - `floordiv`
* `%` - `mod`

All the comparison operators also have corresponding methods.
* `>` - `gt`
* `<` - `lt`
* `>=` - `ge`
* `<=` - `le`
* `==` - `eq`
* `!=` - `ne`

Let's select the undergraduate population column, `UGDS` as a Series, add 100 to it and verify that both the plus operator its corresponding method, `add`, give the same result.

In [29]:
ugds          = college['UGDS']
ugds_operator = ugds + 100
ugds_method   = ugds.add(100)
ugds_operator.equals(ugds_method)

True

#### Calculating the z-scores of each school
Let's do a slightly more complex example. Below, we set the index to be the institution name and then select both of the SAT columns. We remove schools that do not provide these scores with `dropna`.

In [30]:
college_idx = college.set_index('INSTNM')
sats        = college_idx[['SATMTMID','SATVRMID']].dropna()
sats.head()

,SATMTMID,SATVRMID
INSTNM,,
Alabama A & M University,428.0,425.0
University of Alabama at Birmingham,645.0,535.0
University of Alabama in Huntsville,600.0,605.0
Alabama State University,428.0,433.0
The University of Alabama,580.0,585.0


Let's say we are instered in finding the z-score for each college's SAT score. To calculate this, we would need to subtract the mean and divide by the standard deviation. Let's first calculare the mean and standard deviation of each column.

In [32]:
mean = sats.mean()
mean

SATMTMID    555.317924
SATVRMID    559.894566
dtype: float64

In [33]:
std = sats.std()
std

SATMTMID    69.409566
SATVRMID    66.240463
dtype: float64

In [34]:
zscore_operator = (sats - mean) / std
zscore_operator.head()

,SATMTMID,SATVRMID
INSTNM,,
Alabama A & M University,-1.834299,-2.036438
University of Alabama at Birmingham,1.292071,-0.375821
University of Alabama in Huntsville,0.643745,0.680935
Alabama State University,-1.834299,-1.915665
The University of Alabama,0.355600,0.379005


Let's repeat this with their corresponding methods and verify equality.

In [35]:
zscore_methods = sats.sub(mean).div(std)
zscore_operator.equals(zscore_methods)

True